# 条件随机场（CRF）

作者：杨岱川

时间：2019年10月

github：https://github.com/DrDavidS/basic_Machine_Learning

开源协议：[MIT](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/LICENSE)

## 条件随机场简介

条件随机场（Conditional random field, 简称CRF）是一种常用于模式识别和机器学习的统计建模方法，可以用于结构预测等。这是一种给定一组输入随机变量条件下，另一组输出随机变量的条件概率分布模型。本章主要讨论将条件随机场用于标注问题的应用，因此主要讲述线性链（linear chain）条件随机场，这时，问题变成了由输入序列对输出序列预测的判别模型。

李航《统计学习方法（第二版）》中对于CRF的描述过于理论，直接阅读会比较吃力，本章节的目标是对书中难点进行必要补充，让CRF知识点更容易理解。

## 应用背景

### 简单例子

参考阅读[Introduction to Conditional Random Fields](http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/)

不同于《统计学习方法（第二版）》开篇就上公式，我们先通过例子或者例题来理解知识点会更有效：

>假设我们有小明一天中的一些生活照，我们希望用它所代表的活动（吃饭 or 睡觉 or 开车）来标注每张照片，应该怎么做？

最简单的方法就是构建一个图片分类器，比如卷积神经网络或者是别的什么方法等，然后对各种标记的图像进行分类。比如早上6点拍摄的深色图片往往表示小明在睡觉；霓虹灯闪烁的图片可能是小明在跳舞；而汽车图片则表示小明正在开车。

但是如果我们忽略了**顺序**信息，我们很可能会遇到一些问题。

>比如有一张小明**张嘴**的特写照片，那你应该怎么标注？张嘴有可能是“吃饭”，也有可能是“正在唱歌”——但是如果我们手里面有**前一时刻**的照片，比如小明正在做饭或者吃饭，那么这张“张嘴”照片大概率也是在吃饭；而如果前一时刻的照片显示小明拿着话筒在跳舞，那么这张“张嘴”的照片则大概率是在唱歌。

所以，为了让这个照片分类器有更好的表现，在给这一系列照片分类（标注）的时候，我们如果能把相邻时刻（前后均可）照片的标签信息考虑进来就更好啦！这就是CRF的作用。

### 词性标注

词性标注是自然语言处理（简称NLP）中非常常见的问题，就是给一个句子中每一个词标注其词性。比如：

>原句：Bob drank coffee at Starbucks
>
>标注后：Bob (NOUN) drank (VERB) coffee (NOUN) at (PREPOSITION) Starbucks (NOUN)

我们需要针对性地建立一个CRF来标记句子。

## 条件随机场的定义与形式

###  条件随机场的定义

条件随机场是给定随机变量 $X$ 的条件下，随机变量 $Y$ 的马尔可夫随机场。

设 $X$ 与 $Y$ 是随机变量，$P(X|Y)$ 是在给定 $X$ 的条件下 $Y$ 的条件概率分布。若随机变量 $Y$ 构成一个由无向图 $G=(V,E)$ 表示的马尔可夫随机场，即

 $$P(Y_v|X,Y_w,w \ne v)=P(Y_v|X,Y_w,w \sim v)$$
 
对任意结点 $v$ 成立，称条件概率分布 $P(Y|X)$ 为条件随机场。式中 $w \sim v$ 表示在图 $G=(V,E)$ 中与结点 $v$ 有边连接的所有结点 $w$ ， $w \ne v$ 表示结点 $v$ 以外的所有结点，$Y_v$，$Y_u$ 与 $Y_w$ 为节点 $v$，$u$ 与 $w$ 对应的随机变量。

### 线性链条件随机场的定义

我们一般假设 $X$ 和 $Y$ 有相同的图结构，标注问题中正是如此。

![线性链条件随机场](http://images2015.cnblogs.com/blog/779670/201705/779670-20170520235912228-1261738335.png)

线性链条件随机场定义如下：

设 $X=(X_1,X_2,\cdots,X_n),Y=(Y_1,Y_2,\cdots,Y_n)$ 均为线性链表示的随机变量序列，若在给定随机变量序列 $X$ 的条件下，随机变量序列 $Y$ 的条件概率分布构成条件随机场，即**满足马尔可夫性**：

 $$\large P(Y_i|X,Y_1,\cdots,Y_{i-1},Y_{i+1},\cdots,Y_n)=P(Y_i|X,Y_{i-1},Y_{i+1}) $$
 $$\large i=1,2,\cdots,n(在 i=1 和 n 时候只考虑单边) $$
 
则称 $P(X|Y)$ 为线性链条件随机场。在标注问题中，$X$ 表示输入观测序列，$Y$ 表示对应的输出标记序列或状态序列。

#### 例子解释1

回到之前的例子，我们有标注后的句子：

>Bob (NOUN) drank (VERB) coffee (NOUN) at (PREPOSITION) Starbucks (NOUN)

其观测序列 $X$ 就是原句中的每一个单词组成的序列，其状态序列 $Y$ 就是我们对其词性的标记。

$$ 
\begin{cases}
X_1&=\rm{Bob},&\quad Y_1=\rm{NOUN} \\
X_2&=\rm{drank},&\quad Y_2=\rm{VERB} \\
X_3&=\rm{coffee},&\quad Y_3=\rm{NOUN} \\
X_4&=\rm{at},&\quad Y_4=\rm{PREPOSITION} \\
X_5&=\rm{Starbucks},&\quad Y_5=\rm{NOUN}
\end{cases}
$$

注意，因为要求**满足马尔可夫性**，所以我们从公式中可以很直观地看到：

当 $X_3=\rm{coffee}$ 时，$X_3$的词性 $Y_3$ 由 $P(Y_i|X,Y_{i-1},Y_{i+1})$决定，其中参数是：

- 整句话，或者说是整个单词序列 $X$
- 前词的词性 $Y_2=\rm{VERB}$
- 后词的词性 $Y_4=\rm{PREPOSITION}$

### 线性链条件随机场的参数化形式

线性链条件随机场的参数化形式定义如下：

设 $ P(X|Y) $ 为线性链条件随机场，则在随机变量 $X$ 取值为 $x$ 的条件下，随机变量 $Y$ 取值为 $y$的条件概率具有如下形式：

$$\large P(y|x)=\frac{1}{Z(x)}{\rm exp} \left( \sum_{i,k}\lambda_kt_k(y_{i-1},y_i,x,i)+ \sum_{i,l}\mu_ls_l(y_i,x,i)\right)  $$

其中

$$\large Z(x)=\sum_y {\rm exp} \left( \sum_{i,k}\lambda_kt_k(y_{i-1},y_i,x,i)+ \sum_{i,l}\mu_ls_l(y_i,x,i)\right) $$

在上述公式中，

- $t_k$ 和 $s_l$ 是特征函数
- $\lambda_k$ 和 $ \mu_l $ 是对应的权值
- $ Z(x) $ 是规范化因子

求和是在所有可能的输出序列上进行的。

#### 例子解释2

上述公式粗看是非常晦涩难懂的，我们需要使用之前举过的例子来帮助理解。

>Bob (NOUN) drank (VERB) coffee (NOUN) at (PREPOSITION) Starbucks (NOUN)

在**转移特征函数** $t_k(y_{i-1},y_i,x,i)$ 中，比如当前单词是 $X_3=\rm{coffee}$，则

- $y_{i-1}$ 是前一个单词的标签（相邻单词，也就是线性链的由来），这里是 $y_2=\rm{VERB}$；
- $y_{i}$ 是当前单词的标签，这里是 $y_3=\rm{NOUN}$；
- $x$ 是当前句子，就是整个单词序列；
- $i$ 是当前句子中单词的位置，这里 $i=3$。

而权重 $ \lambda_k $则是通过学习（训练）得到的。

同理，**状态特征函数** $s_l$ 及其参数也是如此，其权重 $ \mu_l $则是通过学习（训练）得到的。

### 线性链条件随机场的简化形式

在很多博客论坛或者知乎上，线性链条件随机场的公式要简单一些，这是因为他们采用的是简化形式：

$$
\begin{equation}\begin{split} 
\large P(y|x)
&=\large \frac{1}{Z(x)}{\rm exp}\sum_{k=1}^K w_k \sum_{i=1}^n f_k(y_{i-1},y_i,x,i) \\
&=\large \frac{1}{Z(x)}{\rm exp}\sum^K_{k=1}w_kf_k(y,x)
\end{split}\end{equation}
$$

其中，

- $f_k(y,x)=\sum_{i=1}^n f_k(y_{i-1},y_i,x,i), \quad k=1,2,\cdots,K$ 
- $w_k$ 表示特征 $f_k(y_x)$ 的权值

具体的化简方法暂时不详细推导了，可以参见《统计学习方法（第二版）》第11.2.3节。